In [1]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import gc
import tensorflow as tf
from sklearn.externals import joblib
import numpy as np
from transformers import *
import csv
import base64
import swifter
import pandas as pd
import torch.nn as nn

/root/anaconda3/envs/tianchi_env/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/root/anaconda3/envs/tianchi_env/lib/python3.6/site-packages/swifter/swifter.py:14: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.errors', please update to use 'numba.core.errors' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.errors import TypingError
/root/anaconda3/envs/tianchi_env/lib/python3.6/site-packages/swifter/swifter.py:14: NumbaDeprecationWarning: An import was requested from a module that has m

In [2]:
# config = tf.ConfigProto()
# config.gpu_options.allow_growth=False  # 不全部占满显存, 按需分配
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda:0")
# device = torch.device("cpu")

In [24]:
print('load albert model...')
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertModel.from_pretrained('albert-base-v2')
model = model.to(device)
print('finish to load albert model.')

load albert model...
finish to load albert model.


In [4]:
def convertBoxes(num_boxes, boxes):
    return np.frombuffer(base64.b64decode(boxes), dtype=np.float32).reshape(num_boxes, 4)


def convertFeature(num_boxes, features,):
    return np.frombuffer(base64.b64decode(features), dtype=np.float32).reshape(num_boxes, 2048)


def convertLabel(num_boxes, label):
    return np.frombuffer(base64.b64decode(label), dtype=np.int64).reshape(num_boxes)

In [7]:
def text_and_label_encoder(text):
    input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    with torch.no_grad():
        out = model(input_ids.cuda())[1][0]
    return np.array(out.cpu())

# train data

In [2]:
def convertBoxes(num_boxes, boxes):
    return np.frombuffer(base64.b64decode(boxes), dtype=np.float32).reshape(num_boxes, 4)


def convertFeature(num_boxes, features,):
    return np.frombuffer(base64.b64decode(features), dtype=np.float32).reshape(num_boxes, 2048)


def convertLabel(num_boxes, label):
    return np.frombuffer(base64.b64decode(label), dtype=np.int64).reshape(num_boxes)

device = torch.device("cuda")
print('load albert model...')
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertModel.from_pretrained('albert-base-v2',output_hidden_states=True)
model = model.to(device)
print('finish to load albert model.')

# output mean pool of last second hidden layer state
def text_and_label_encoder(text):
    input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    with torch.no_grad():
        out = model(input_ids.cuda())[2][-2].mean(dim=1)[0]
    return np.array(out.cpu())

TRAIN_PATH = '../../tianchi_data/train.tsv'

print("starting")

LEN = 0
# 一次读100000条训练数据
train_data = pd.read_csv(TRAIN_PATH, sep='\t', chunksize=100000, quoting=csv.QUOTE_NONE)
for train in train_data:
    LEN += len(train)
    print(LEN)
    albert_embedded=pd.DataFrame()
    text_train = train.swifter.apply(lambda x: text_and_label_encoder(x['query']), axis=1)
    text_train.reset_index(drop=True, inplace=True)
    albert_embedded=pd.concat((albert_embedded, text_train),axis=0)

    print('_'*40)
    
    joblib.dump(albert_embedded, 'avgpool_albert_embedded_'+str(LEN))
    print('avgpool_albert_embedded_{} is saved!'.format(LEN))
    del train, text_train, albert_embedded
    gc.collect()

load albert model...
finish to load albert model.
starting
1100000



________________________________________
avgpool_albert_embedded_1100000 is saved!
1200000



________________________________________
avgpool_albert_embedded_1200000 is saved!
1300000



________________________________________
avgpool_albert_embedded_1300000 is saved!
1400000



________________________________________
avgpool_albert_embedded_1400000 is saved!
1500000


...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




________________________________________
avgpool_albert_embedded_2700000 is saved!
2800000



________________________________________
avgpool_albert_embedded_2800000 is saved!
2900000



________________________________________
avgpool_albert_embedded_2900000 is saved!
3000000



________________________________________
avgpool_albert_embedded_3000000 is saved!


# valid data

In [3]:
def convertBoxes(num_boxes, boxes):
    return np.frombuffer(base64.b64decode(boxes), dtype=np.float32).reshape(num_boxes, 4)


def convertFeature(num_boxes, features,):
    return np.frombuffer(base64.b64decode(features), dtype=np.float32).reshape(num_boxes, 2048)


def convertLabel(num_boxes, label):
    return np.frombuffer(base64.b64decode(label), dtype=np.int64).reshape(num_boxes)

device = torch.device("cuda:0")
print('load albert model...')
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertModel.from_pretrained('albert-base-v2',output_hidden_states=True)
model = model.to(device)
print('finish to load albert model.')

# output mean pool of last second hidden layer state
def text_and_label_encoder(text):
    input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    with torch.no_grad():
        out = model(input_ids.cuda())[2][-2].mean(dim=1)[0]
    return np.array(out.cpu())

TRAIN_PATH = '../../tianchi_data/valid.tsv'

print("starting")
albert_embedded=pd.DataFrame()
valid = pd.read_csv(TRAIN_PATH, sep='\t', quoting=csv.QUOTE_NONE)
text_valid = valid.swifter.apply(lambda x: text_and_label_encoder(x['query']), axis=1)
albert_embedded=pd.concat((albert_embedded, text_valid),axis=0)

print('_'*40)
gc.collect()

joblib.dump(albert_embedded, 'avgpool_albert_embedded_valid')
print('avgpool_albert_embedded_valid is saved!')

load albert model...
finish to load albert model.
starting



________________________________________


NameError: name 'LEN' is not defined

# test data

In [2]:
def convertBoxes(num_boxes, boxes):
    return np.frombuffer(base64.b64decode(boxes), dtype=np.float32).reshape(num_boxes, 4)


def convertFeature(num_boxes, features,):
    return np.frombuffer(base64.b64decode(features), dtype=np.float32).reshape(num_boxes, 2048)


def convertLabel(num_boxes, label):
    return np.frombuffer(base64.b64decode(label), dtype=np.int64).reshape(num_boxes)

device = torch.device("cuda")
print('load albert model...')
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertModel.from_pretrained('albert-base-v2',output_hidden_states=True)
model = model.to(device)
print('finish to load albert model.')

# output mean pool of last second hidden layer state
def text_and_label_encoder(text):
    input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    with torch.no_grad():
        out = model(input_ids.cuda())[2][-2].mean(dim=1)[0]
    return np.array(out.cpu())

TRAIN_PATH = '../../tianchi_data/testB.tsv'

print("starting")
albert_embedded=pd.DataFrame()
test = pd.read_csv(TRAIN_PATH, sep='\t', quoting=csv.QUOTE_NONE)
text_test = test.swifter.apply(lambda x: text_and_label_encoder(x['query']), axis=1)
albert_embedded=pd.concat((albert_embedded, text_test),axis=0)

print('_'*40)
gc.collect()

joblib.dump(albert_embedded, 'avgpool_albert_embedded_testB')
print('avgpool_albert_embedded_testB is saved!')

load albert model...
finish to load albert model.
starting



________________________________________
avgpool_albert_embedded_testB is saved!
